# Setup

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from utilities import *
from config import *
from dataloading import *
from tqdm import tqdm
from transformer import *
import os

In [ ]:
for x in train_dataloader:
    data, train = x
    break

In [ ]:
data[:10]

In [ ]:
train[:10]

In [ ]:
model = BigramLanguageModel()

In [ ]:
# setup the model
model = BigramLanguageModel()

# cuda? (gpu)
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"

# send to gpu (maybe)
model = nn.DataParallel(model)
model = model.to(device)

# optionally: load the model
filename = PATH + "/model/" + MODELNAME + ".pth"
if os.path.isfile(filename):
    model.load_state_dict(torch.load(filename, map_location=torch.device(device)))

os.path.isfile(filename)

In [ ]:
import pyperclip

def np_to_mathematica(array, copy=True):
    formatted = str(array.tolist()).replace("[", "{").replace("]", "}")
    return formatted

In [ ]:
pyperclip.copy(np_to_mathematica(embedding_pca))
print("Copied!")

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

embedding = np.array(model.module.token_embedding_table.weight.cpu().detach().numpy())
pos_embedding = np.array(model.module.position_embedding.weight.cpu().detach().numpy())

pca = PCA(n_components=3)
pca.fit(embedding)

embedding_pca = pca.transform(embedding)

pca = PCA(n_components=2)
pca.fit(pos_embedding)

pos_embedding_pca = pca.transform(pos_embedding)

In [ ]:
import plotly.express as px

similarity = []

for x in embedding:
    row = []
    for y in embedding:
        row.append(np.dot(x, y))
    similarity.append(row)

px.imshow(similarity)

In [ ]:
np.dot(pos_embedding[MAX_LENGTH], embedding[START_PREDICTION_TOKEN])

In [ ]:
convert_to_transposition(13)

In [ ]:
threshold = 30

for pos1, x in enumerate(embedding):
    for pos2, y in enumerate(embedding):
        if np.dot(x, y) > threshold and pos1 != pos2:
            print("x", pos1, "y", pos2, "dot", np.dot(x, y))

In [ ]:
px.imshow(model.module.position_embedding.weight.detach())

In [ ]:
torch.no_grad()
model.eval()

In [ ]:
targets

In [ ]:
# calculate validation stats
criterion = nn.CrossEntropyLoss()

total_accuracy = 0.0
total_loss = 0.0

num_batches = 0

print("Evaluating...")
for inputs, targets in tqdm(val_dataloader):
    outputs = model(inputs)

    # calculate the val accuracy
    accuracy = calculate_accuracy(outputs, targets)
    total_accuracy += accuracy

    # Calculate the val loss
    loss = criterion(outputs, targets)
    total_loss += loss.item()
    num_batches += 1

average_accuracy = total_accuracy / num_batches
val_loss = total_loss / num_batches

In [ ]:
val_loss

In [ ]:
vocab_size

In [ ]:
output = model(data)

In [ ]:
train

In [ ]:
train.shape

In [ ]:
output.shape

In [ ]:
output[1]

In [ ]:
model.get_device()

In [ ]:
dev

In [ ]:
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

In [ ]:
conver

In [ ]:
seq = [0,0,2,20,1,0]
get_permutation(seq)

In [ ]:
model.module.generate(seq)

In [ ]:
model.eval()

# use gpu for processing
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

# create an initial input
input_tensor = torch.ones(block_size, dtype=int).to(dev)
input_tensor *= TO_PREDICT_TOKEN
input_tensor[:len(seq)] = torch.tensor(seq, dtype=int).to(dev)
input_tensor[len(seq)] = START_PREDICTION_TOKEN

In [ ]:
input_tensor.unsqueeze(0)

In [ ]:
model(input_tensor.unsqueeze(0))

In [ ]:
torch.argmax(model(input_tensor.unsqueeze(0)), dim=1)

In [ ]:
prediction_tensor = torch.zeros(block_size, dtype=int).to(dev)

In [ ]:
sequence = [1,2,3,1,2,3]

In [ ]:
prediction_tensor[:len(sequence)] = torch.tensor(sequence, dtype=int).to(dev)

In [ ]:
prediction_tensor

In [ ]:
input_tensor = torch.ones(block_size, dtype=int).to(dev)

In [ ]:
input_tensor *= TO_PREDICT_TOKEN

In [ ]:
input_tensor